# Capa

<img src="./images/ESTB.png"
     alt="ESTBarreiro"
     style="float: left; margin-right: 10px;" />
<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>
- ESTBarreiro - IPS Bioinformatica - Janeiro 2023
- Ano 3, semestre 1
- UC: Big Data
- Duarte Valente - 202000053
- Guilherme Sá   - 202000201
- Docentes: Raquel Barreira e António Gonçalves

# Introduçao

## Objetivos deste trabalho:
- Este projeto tem como principal objetivo dar-nos a conhecer e utilizar o Spark no contexto de Machine Leaning.
- Aqui vamos usar tecnicas de machine learning para analisar uma grande quantidade de dados e prever resultados para um determinado problema, assim como usar o spark para ler e apresentar resultados relativamente aos modelos de ML.
- Tudo isto tem o intuito de simular um ambiente de grande escala e alta performance.
## Introducao ao dataset utilizado
- Nome: SparkML - Credit Card Fraud - KMA
- Nome do dataset: creditcard.csv
- O nosso dataset pode ser encontrado [aqui](https://www.kaggle.com/code/ukveteran/sparkml-credit-card-fraud-jma/data)
- Este conjunto de dados é, nada mais nada menos que informaçoes sobre transaçoes de credit cards.
- Com isto, o objetivo do trabalho será prever, de acordo com as caracteristicas de uma transaçao, se a mesma é ou nao fraude.

# Projeto

## Instalar os packaged necessarios

In [2]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [51]:
# Spark Session imports
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
# Spark ML imports
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
# other imports 
import matplotlib as plt

## Criar Ambiente do Spark
- Aqui estamos a criar um ambiente de spark com o nome de CreditcardPrediction e com o computador master
- Quando se trabalha com o Spark temos de criar uma Sessao SparkSession para gerir todas as configuraçoes do nosso ambienter de spark.

In [25]:
conf= SparkConf().setAppName('CreditCardPrediction').setMaster('local[*]')
sc= SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)
spark

## Importar os dados (creditcard.csv)

### Importar os dados por ficheiro

In [42]:
df = spark.read.csv("creditcard.csv", header=True, inferSchema=True)

## Visualisar dados
Vamos simplesmente observar os dados, com o pandas, com o spark e ver a dimanesao e disparidade dos dados

In [7]:
df.toPandas().head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### Descriçao das caracteristicas do Dataset:
- Time: Numero de segundos passados entre a transaçao em causa e a primeira transaçao do dataset.
- V (1-28): Sao dados dados que foram alterados propositadamente para que informaçoes dos utilizadores fossem protegidas.
- Amount: Quantidade de dinheiro transferido
- Class: Verificaçao se a transaçao era ou nao fraude, 1= fraude e 0 para todos os outros casos.

In [8]:
df.show(5)

+----+------------------+-------------------+----------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------+-----+
|Time|                V1|                 V2|              V3|                V4|                 V5|                 V6|                 V7|                V8|                V9|                V10|               V11|               V12|               V13|               V14|               V15|               V16|               V17|                V18|               V19|                V20|                 V21|                V22|     

In [26]:
counts = df.groupBy("Class").count()
counts.show()

+-----+------+
|Class| count|
+-----+------+
|    1|   492|
|    0|284315|
+-----+------+



In [27]:
print(f"Numero de Linhas: {df.count()}")
print(f"Numero de Colunas:{len(df.columns)}")
print("Esquema do DataFrame:")
df.printSchema()

Numero de Linhas: 284807
Numero de Colunas:31
Esquema do DataFrame:
root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullabl

- Aqui Conseguimos ver que o nosso dataset tem 284807 items e 31 caracteristicas, assim como podemos verificar que todas as caracteristicas podem aceitar valores nulos.

## Sumarizaçao dos Dados
- Teem uma dimensao de 284807 entidades e 31 caracteristicas.
- Teem uma grande amostra de casos de nao-fraude (492) e muito poucos de fraude (284315), o que pode levar a problemas na detecao de fraunde por nao ter amostra suficiente

## Gráficos:

### graficos para todas as caracteristicas e a sua distribuiçao

In [44]:
num_cols = [i for i in df.columns]
df_num = df.select(num_cols)

In [ ]:
df_num.toPandas().hist(figsize=[15,15],bins=30, color='orange', histtype='bar', ec='black')


: 

#### Breve explicaçao dos graficos:

- Estes histogramas estao a ser apresentados com o objetivo de mostrar a distribuiçao dos valores pelo count de um determinado valor.

#### Breve analise dos graficos:

- Time - Nao ha muito que se possa retirar deste histograma, apenas que tem valores significativos a correr ao longo de todo o percurso. Significa que nao houve um tempo relevante sem haver nenhuma transaçao
- Vs - Nao conseguimos tirar informaçao relevante por nao se saber a natureza dos dados.
- Amount - Conseguimos verificar que a grande maioria das transaçoes está concentrada em valores de até 5000
- Class - Conseguimos verificar que existe uma concentraçao muito maior no valor 0, correspondente a "Nao é fraude". Isto é bom no entanto por haver uma grande disparidade na quantidade de dados entre valores positivos e negativos poderá levar a problemas na nossa previsao de resultados com machine learning.

## Tratamento dos dados

### Pré-processamento:

#### Vamos começar por verificar a existencia de valores nulos.

In [33]:
null_values = []
for col in df.columns:
    null_values.append((col, df.filter(df[col].isNull()).count()))

In [36]:
col_count = 0
null_col_count = 0
for col, value in null_values:
    col_count = col_count + 1
    if value == 0:
        null_col_count = null_col_count + 1
    else:
        print(f"{col} : {value} missing values")
if col_count == null_col_count:
    print("Nao ha valores nulos")

Nao ha valores nulos


### Tranformaçao de dados

- Aqui vamos pegar em todas as caracteristicas dos dados exceto a variavel target e criar um objeto vetor para o item. Vamos fazer isto para obter um resultado de: 
- [Caracteristica 1 | C2 | C3 | C4 | ... | Class] para:
- [C1 | Class]

In [38]:
cols = df.columns
cols.remove('Class')

Primeiro adicionamos todas as colunas a uma variavel e removemos apenas a ultima coluna.

In [39]:
assembler = VectorAssembler(inputCols= cols, outputCol= 'features')

Em seguida fomos "colar" as caracteristicas dentro desta nova variavel e dar o nome ao conjunto de "features"

In [40]:
df = assembler.transform(df)

Agora vamos visualizar os dados depois de serem tratados, e vamos poder observar que de 30 passamos a ter apenas 2 colunas e vamos usar a coluna das features para prever a da class, uma vez que a features nao e nada mais nada menos que todas as outras 29 caracteristicas juntas.

In [41]:
df = df.select(['features', 'Class'])
df.show(5)

+--------------------+-----+
|            features|Class|
+--------------------+-----+
|[0.0,-1.359807133...|    0|
|[0.0,1.1918571113...|    0|
|[1.0,-1.358354061...|    0|
|[1.0,-0.966271711...|    0|
|[2.0,-1.158233093...|    0|
+--------------------+-----+
only showing top 5 rows



## Aplicar Modelos de Machine Learning
Neste trabalho vao ser usados 2 modelos de machine learning, a Regressao logistica e o random forest. Estes modelos foram escolhidos por serem bons candidatos a tratar de classificacao binaria, que era o caso do nosso dataset.

### Defenir Test/Split
Neste caso vamos fazer o split a 70% dos dados para treino e os restantes 30% para teste

In [15]:
training, testing= df.randomSplit([0.7, 0.3])

### Modelo - Regressao Logistica

#### treino e teste do modelo

In [16]:
lr = LogisticRegression(labelCol='Class', featuresCol='features')
model= lr.fit(training) # fit dos dados de treino
predictions = model.transform(testing) # teste do modelo com os dados de teste

#### Verificar resultados do teste

In [17]:
evaluator = BinaryClassificationEvaluator(labelCol='Class', rawPredictionCol='rawPrediction')
accuracy = evaluator.evaluate(predictions)
accuracy

0.9697310527580402

Com isto temos um modelo de previsao a usar regressao logistica em que obtivemos uma precisao de aproximadamente 98%

#### tabela com a previsao

In [18]:
predictions.toPandas()

,features,Class,rawPrediction,probability,prediction
0,"[10.0, 1.44904378114715, -1.17633882535966, 0....",0,"[9.676652548297383, -9.676652548297383]","[0.9999372727927788, 6.272720722122482e-05]",0.0
1,"[12.0, -2.7918547659339, -0.327770756658658, 1...",0,"[6.703446032088004, -6.703446032088004]","[0.9987748254529829, 0.0012251745470170938]",0.0
2,"[16.0, 0.694884775607337, -1.36181910308009, 1...",0,"[8.88536619773032, -8.88536619773032]","[0.9998616196614877, 0.0001383803385123228]",0.0
3,"[18.0, 0.247491127783665, 0.277665627353681, 1...",0,"[7.277680717978781, -7.277680717978781]","[0.9993096910212618, 0.0006903089787382299]",0.0
4,"[22.0, -1.94652513121534, -0.0449005054418194,...",0,"[11.623858808409619, -11.623858808409619]","[0.9999910500960569, 8.949903943067383e-06]",0.0
...,...,...,...,...,...
85219,"[172769.0, -1.02971891697297, -1.1106695791175...",0,"[8.154066803077358, -8.154066803077358]","[0.9997125191547755, 0.0002874808452244526]",0.0
85220,"[172770.0, -0.446950896052929, 1.3022123695422...",0,"[7.749044063836653, -7.749044063836653]","[0.9995690313142689, 0.0004309686857311146]",0.0
85221,"[172770.0, 2.00741802782449, -0.28023496951448...",0,"[9.212862424238045, -9.212862424238045]","[0.9999002618361497, 9.973816385033807e-05]",0.0
85222,"[172774.0, -0.724122855662227, 1.4852156432536...",0,"[8.46793988892914, -8.46793988892914]","[0.9997899468500234, 0.00021005314997657454]",0.0


### Modelo - Random forest

#### treino e teste do modelo
Neste caso, estamos a fazer um modelo com uma profuncidade maxima de 5 para evitar overfitting.

In [19]:
rf = RandomForestClassifier(labelCol="Class", featuresCol="features", maxDepth= 5)
model = rf.fit(training)
predictions = model.transform(testing) # teste do modelo com os dados de teste

#### verificar resultados do teste

In [20]:
evaluator = BinaryClassificationEvaluator(labelCol='Class', rawPredictionCol='rawPrediction')
accuracy = evaluator.evaluate(predictions)
accuracy

0.9670332541925177

Com isto temos um modelo de previsao a usar o Random Forest em que obtivemos uma precisao de aproximadamente 96%

#### tabela com a previsao

In [21]:
predictions.toPandas()

,features,Class,rawPrediction,probability,prediction
0,"[10.0, 1.44904378114715, -1.17633882535966, 0....",0,"[19.9953404518565, 0.0046595481434967955]","[0.9997670225928251, 0.0002329774071748398]",0.0
1,"[12.0, -2.7918547659339, -0.327770756658658, 1...",0,"[19.993730391353413, 0.006269608646583751]","[0.9996865195676709, 0.00031348043232918763]",0.0
2,"[16.0, 0.694884775607337, -1.36181910308009, 1...",0,"[19.9953404518565, 0.0046595481434967955]","[0.9997670225928251, 0.0002329774071748398]",0.0
3,"[18.0, 0.247491127783665, 0.277665627353681, 1...",0,"[19.9953404518565, 0.0046595481434967955]","[0.9997670225928251, 0.0002329774071748398]",0.0
4,"[22.0, -1.94652513121534, -0.0449005054418194,...",0,"[19.9953404518565, 0.0046595481434967955]","[0.9997670225928251, 0.0002329774071748398]",0.0
...,...,...,...,...,...
85219,"[172769.0, -1.02971891697297, -1.1106695791175...",0,"[19.9927327368346, 0.007267263165397305]","[0.9996366368417302, 0.0003633631582698653]",0.0
85220,"[172770.0, -0.446950896052929, 1.3022123695422...",0,"[19.9953404518565, 0.0046595481434967955]","[0.9997670225928251, 0.0002329774071748398]",0.0
85221,"[172770.0, 2.00741802782449, -0.28023496951448...",0,"[19.9953404518565, 0.0046595481434967955]","[0.9997670225928251, 0.0002329774071748398]",0.0
85222,"[172774.0, -0.724122855662227, 1.4852156432536...",0,"[19.995060753771806, 0.004939246228189483]","[0.9997530376885905, 0.0002469623114094742]",0.0


# Conclusao 
Neste trabalho foi utilizado o PySpark para carregar, processar e analisar um dataset. Em seguida foi utilizado o MLlib para treinar e avaliar modelos de machine learning. Como modelos foram usados a regressao logistica e o Random Forest. Ambos os modelos optiveram um desempenho que consideramos adequado na previsao de se havia fraude ou nao na transacao. Estamos satisfeitos com os resultados obtidos. Podendo até dizer que correu melhor que o esperado dada a disparidade entre os dados da variavel target.